<a href="https://colab.research.google.com/github/samgregson/GHPT-colab-experiments/blob/main/Grasshopper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
install openai and setup API key

This notebook works both in a Colab environment and on local machine

Colab:
- API key must be saved in Colab sectrets as OPENAI_API_KEY

Local:
- API key must be defined in the .env file (refer to example.env)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
IN_COLAB

False

In [2]:
if IN_COLAB:
    %pip install openai
    %pip install requests

if IN_COLAB:
    import os
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "GHPT_Instructor"

### load files into Colab and install package

NOTE: you may need to refresh your Colab files directory to see changes

In [3]:
if IN_COLAB:
    # remove the existing directory
    import shutil
    shutil.rmtree('/content/GHPT-experiments/', ignore_errors=True)
    %git clone "https://github.com/samgregson/GHPT-experiments"
    %pip install -e /content/GHPT-experiments/
    # add the modules to the search path
    import site
    site.main()

Load the correct SSL certificates

In [ ]:
if not IN_COLAB:
    import os
    import ssl
    context = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))

Load the correct SSL certificates

In [ ]:
if not IN_COLAB:
    import os
    import ssl
    context = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))

### OpenAI

In [4]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
if not IN_COLAB:
    from patch_openai.patch_openai import patch_openai
    client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

### test connection

In [5]:
# model: str = "gpt-3.5-turbo-1106"
# temperature: float = 0

# prompt="say this is a test"
# system_prompt=""

# completion = call_openai(prompt=prompt, system_prompt=system_prompt, model=model, temperature=temperature)
# result = completion.choices[0].message.content
# print(result)

# GHPT

### Import prompts

In [7]:
from prompts.generate_script import system_prompt, prompt_template

# replace all { with {{ and } with }} to escape the curly braces
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

## Baseline Call

In [10]:
from langsmith import traceable

# create the openai api call function
@traceable
def call_openai(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    if system_prompt == "":
        messages=[
            {"role": "user", "content": prompt}
        ]
    else:
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]

    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages
    )

    return completion

In [12]:
model: str = "gpt-3.5-turbo-1106"
# model: str = "gpt-3.5-turbo"
# model = "gpt-4"
temperature: float = 0

prompt=system_prompt + prompt_template.format(QUESTION="create a sphere")

completion = call_openai(prompt=prompt, system_prompt="", model=model, temperature=temperature)
result = completion.choices[0].message
print(result.content)

{
	"Advice": "Remember to set the radius of the sphere using a number slider",
	"Additions": [
		{
			"Name": "Sphere",
			"Id": 1
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"value": "0..10..50"
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 1,
				"ParameterName": "Radius"
			},
			"From": {
				"Id": 2,
				"ParameterName": "Number"
			}
		}
	]
}
